In [18]:
import requests
import numpy as np
import glob
import xlrd
import os
import sqlite3
from runSQL import *

def getFilesNames ():
    print("..................................................")
    print("Getting the name of the ANH files from the webpage")
    print("..................................................")
    url = 'http://www.anh.gov.co/estadisticas-del-sector/sistemas-integrados-operaciones/estad%C3%ADsticas-producci%C3%B3n'
    
    html = requests.get(url)
    with open('file.txt', 'wb') as wr: 
        wr.write(html.content)
    wr.close()

    read = open("file.txt","r+",encoding="utf-8")

    htmlExcel = ""
    for line in read:
        xlsx = ".xlsx"
        if xlsx in line:
            htmlExcel = line

    htmlExcel.replace(">", " ")
    htmlExcel.replace("<", " ")
    listaExcel = []

    separacionExcel= np.array(htmlExcel.split(" "))
    for i in range(0,len(separacionExcel)):
        if "href" in separacionExcel[i]:
            l = separacionExcel[i].split("\"")
            listaExcel.append("http://www.anh.gov.co"+l[1])
    read.close()
    os.remove('file.txt')
    return listaExcel

In [19]:
def downloadFiles (listaExcel):
    print("downloading files... this might take some minutes")
    for i in range(0, len(listaExcel)-1):
        dls = listaExcel[i]
        resp = requests.get(dls)
        name = dls.split("/")
        nombre = ""
        for j in range(0, len(name)):
            if ".xls" in name[j] or ".XLS" in name[j]:
                nombre = name[j].split(".xls")[0]
                nombre = nombre.replace("%c3", "on")
                nombre = nombre.replace("b3n%", "")
                nombre = nombre.replace("%","-")
                nombre = nombre.replace(".","")
                nombre = nombre.replace("2020","20")

                if len(name[j].split(".xls"))< 2:
                    nombre = nombre + ".xlsx"
                else:
                    if "x" in name[j].split(".xls")[1]:
                        nombre = nombre + ".xlsx"
                    else:
                        nombre = nombre + ".xls"
        
        output = open(nombre, 'wb')
        output.write(resp.content)
        output.close()
    print("Download complete!")
        

In [54]:
def getGasOilFileNames(result):
    local = ""
    if result == False:
        local = "Files\\"
    directorio = local+"*.xlsx"
    directorio2 = local + "*.xls"
    print(local)
    print(directorio)
    listnamefiles = [f for f in glob.glob(directorio)]
    listnamefiles2 = [f for f in glob.glob(directorio2)]
    listnameGas=[]
    listnameOil=[]
    listnameAll = []

    for i in range(0,len(listnamefiles)):
        if "Gas" in listnamefiles[i] or "gas" in listnamefiles[i]:
            listnameGas.append(listnamefiles[i])
        elif "Crudo" in listnamefiles[i] or "crudo" in listnamefiles[i]:
            listnameOil.append(listnamefiles[i])
        listnameAll.append(listnamefiles[i])
        
    for i in range(0,len(listnamefiles2)):
        if "Gas" in listnamefiles2[i] or "gas" in listnamefiles2[i]:
            listnameGas.append(listnamefiles2[i])
        elif "Crudo" in listnamefiles2[i] or "crudo" in listnamefiles2[i]:
            listnameOil.append(listnamefiles2[i])
        listnameAll.append(listnamefiles2[i])
    return listnameGas, listnameOil, listnameAll 


In [21]:


def encontrarEncabezado(sheet):
    
    encabezado = []
    indiceE = 0
    for i in range(0, 100):
        if sheet.row_values(i)[0]!=''and sheet.row_values(i)[1]!='':
            indiceE = i
            break
    for i in range(0, len(sheet.row_values(indiceE))):
        t = sheet.row_values(indiceE)
       
        if t[i] == '':
            break
        else:
            encabezado.append(t[i])
    
    return encabezado, indiceE
    
    
def encontrarAnio(sheet, indiceE):
    
    encontrado = False
    i = 0
    j = 0
    valor= ""
    while encontrado == False:

        valor = sheet.row_values(i)[j]
        if "20" in valor:
            valor = valor.split("20")[1]
            valor = "20"+valor
            valor = valor.replace("*","")
            if valor == "20":
                valor = "2020"
            encontrado= True
            break
        i+=1
        if i == indiceE:
            j+=1
            i=0
            
    return valor

def orgazinarLosArchivosOil(listnameOil):
    listAnio = []
    listEncabezado = []
    for i in range(0, len(listnameOil)):
        loc = listnameOil[i]
        wb = xlrd.open_workbook(loc)
        sheet = wb.sheet_by_index(0)
        sheet.cell_value(0, 0)

        encabezado, indiceE = encontrarEncabezado(sheet)
        anio = encontrarAnio(sheet, indiceE)
        siHay = 0
        for j in range(0, len(listAnio)):
            if anio == listAnio[j][0]:
                if len(encabezado)> len(listEncabezado[j][0]):
                    listAnio[j] = [anio,i]
                    listEncabezado[j] = [encabezado, indiceE]
                siHay = 1
        if siHay == 0:
            listAnio.append([anio, i])
            listEncabezado.append([encabezado, indiceE])
    return listAnio, listEncabezado

def guardarDatosOil(listnameOil):
    anio, encabezado = orgazinarLosArchivosOil(listnameOil)
    print("starting to save the data in the database this might take some minutes")
    for i in range(0,len(anio)):
        indice = anio[i][1]
        year = anio[i][0]
        loc = listnameOil[indice]
        print("saving: ",loc)
        wb = xlrd.open_workbook(loc)
        sheet = wb.sheet_by_index(0)
        sheet.cell_value(0, 0)
      
        for m in range(encabezado[i][1]+1, sheet.nrows):
            valor = sheet.row_values(m)
           
            if valor[0] == "" or valor[1] == "" or valor[2] == "" or "total" in valor[0].lower():
                break
            insertarTablaCrudo(valor,encabezado[i][0], anio[i][0])
        print("finish with: ", loc)

In [22]:

conn = sqlite3.connect("hackathon.db")
c = conn.cursor()

def create_tables():
    c.execute("CREATE TABLE IF NOT EXISTS CampoCrudo(id INTEGER PRIMARY KEY, campo TEXT, year INTEGER, departamentoID INTEGER, municipioID INTEGER, operadoraID INTEGER, cuencaID INTEGER, contradoID INTEGER, datosID INTEGER )")
    c.execute("CREATE TABLE IF NOT EXISTS Departamento(id INTEGER PRIMARY KEY, nombre TEXT)")
    c.execute("CREATE TABLE IF NOT EXISTS Municipio(id INTEGER PRIMARY KEY, nombre TEXT)")
    c.execute("CREATE TABLE IF NOT EXISTS Empresa(id INTEGER PRIMARY KEY, nombre TEXT)")
    c.execute("CREATE TABLE IF NOT EXISTS Cuenca(id INTEGER PRIMARY KEY, nombre TEXT)")
    c.execute("CREATE TABLE IF NOT EXISTS Contrato(id INTEGER PRIMARY KEY, nombre TEXT)")
    c.execute("CREATE TABLE IF NOT EXISTS Data(id INTEGER PRIMARY KEY, enero REAL, febrero REAL, marzo REAL, abril REAL, mayo REAL, junio REAL, julio REAL, agosto REAL, septiembre REAL, octubre REAL, noviembre REAL, diciembre REAL)")
    c.execute("CREATE TABLE IF NOT EXISTS CampoGas(id INTEGER PRIMARY KEY, campo TEXT, year INTEGER, departamentoID INTEGER, municipioID INTEGER, operadoraID INTEGER, cuencaID INTEGER, contradoID INTEGER, datosID INTEGER, datosMesID INTEGER, mes TEXT)")
    conn.commit()
    
def insertarTablaCrudo(valor, encabezado, year):
    campo = ""
    departamento = ""
    municipio = ""
    operadora = ""
    cuenca = ""
    contrato = ""
    arregloMes = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    termino = False
    for i in range(0,len(encabezado)):
        if valor[i]=="":
            value = "0"
        else:
            value = valor[i]
            
          
        if "departamento" == encabezado[i].lower():
            departamento = value.lower()
        elif "campo" == encabezado[i].lower():
            campo = value.lower()
        elif "municipio" == encabezado[i].lower():
            municipio = value.lower()
        elif "operadora" == encabezado[i].lower() or "empresa" == encabezado[i].lower():
            operadora = value.lower()           
        elif "cuenca" == encabezado[i].lower():
            cuenca = value.lower()
        elif "contrato" == encabezado[i].lower():
            contrato = value.lower()
        elif "enero"== encabezado[i].lower() or "ene" in encabezado[i].lower():
            arregloMes[0] = float(value)
        elif "febrero"== encabezado[i].lower() or "feb" in encabezado[i].lower():
            arregloMes[1] = float(value) 
        elif "marzo"== encabezado[i].lower():
            arregloMes[2] = float(value)
        elif "abril"== encabezado[i].lower():
            arregloMes[3] = float(value)
        elif "mayo"== encabezado[i].lower():
            arregloMes[4] = float(value)
        elif "junio"== encabezado[i].lower():
            arregloMes[5] = float(value)
        elif "julio"== encabezado[i].lower():
            arregloMes[6] = float(value)
        elif "agosto"== encabezado[i].lower():
            arregloMes[7] = float(value)
        elif "septiembre"== encabezado[i].lower():
            arregloMes[8] = float(value)
        elif "octubre"== encabezado[i].lower():
            arregloMes[9] = float(value)
        elif "noviembre"== encabezado[i].lower():
            arregloMes[10] = float(value)
        elif "diciembre"== encabezado[i].lower():
            arregloMes[11] = float(value)
        elif "concatenar" == encabezado[i].lower():
            print("concatenar")
        else: 
            print("no")
            print(encabezado[i].lower())
            print(valor[i])
    
    if termino == True:
        print(termino) 
    else:
        conn = sqlite3.connect("hackathon.db")
        c = conn.cursor()

        idD = insertarDepartamento(departamento)
        idM = insertarMunicipio(municipio)
        idE = insertarEmpresa(operadora)
        idC = insertarCuenca(cuenca)
        idCo = insertarContrato(contrato)
        idDa = insertarData(arregloMes)
        insertarCampoCrudo(idD, idM, idE, idC, idCo, idDa, campo, year)

def quitarAcento(valor):
    v = valor.replace("á","a")
    v2 = v.replace("é","e")
    v3 = v2.replace("í","i")
    v4 = v3.replace("ó", "o")
    v5 = v4.replace("ú", "u")
    return v5

def limpiar(valor):

    v = valor.replace("/"," ")
    v2 = v.replace("-"," ")
    v3 = v2.replace("  ", " ")
    v5 = v3.replace("  ", " ")
    v6 = v5.replace("  ", " ")
    v7 = v6.replace(".", "")
    
    v4 = ""
    if len(v7)!=0:
        if " " == v7[len(v7)-1]:
            for i in range(0, len(v7)-1):
                v4+=v7[i]
        else:
            v4 = v7
    else:
        v4 = v7
    
    return v4

def insertarDepartamento(departamento):
    c.execute("SELECT * FROM Departamento")
    existe = False
    idD = 0
    departamento = quitarAcento(departamento)
    departamento = limpiar(departamento)
    for row in c.fetchall():
        if departamento == row[1]:
            existe =True
            idD = row[0]
            break
        idD = row[0]+1
    if existe == False:
        c.execute("INSERT INTO Departamento(id, nombre) VALUES (?, ?)", (idD, departamento))
        conn.commit()
    return idD

def insertarMunicipio(municipio):
    c.execute("SELECT * FROM Municipio")
    existe = False
    idM = 0
    municipio = quitarAcento(municipio)
    municipio = limpiar(municipio)
    for row in c.fetchall():
        if municipio == row[1]:
            existe =True
            idM = row[0]
            break
        idM = row[0]+1
    if existe == False:
        c.execute("INSERT INTO Municipio(id, nombre) VALUES (?, ?)", (idM, municipio))
        conn.commit()
    return idM

def insertarEmpresa(empresa):
    c.execute("SELECT * FROM Empresa")
    existe = False
    idE = 0
    empresa = quitarAcento(empresa)
    empresa = limpiar(empresa)
    for row in c.fetchall():
        if empresa == row[1]:
            existe =True
            idE = row[0]
            break
        idE = row[0]+1
    if existe == False:
        c.execute("INSERT INTO Empresa(id, nombre) VALUES (?, ?)", (idE, empresa))
        conn.commit()
    return idE

def insertarCuenca(cuenca):
    c.execute("SELECT * FROM Cuenca")
    existe = False
    idC = 0
    cuenca = quitarAcento(cuenca)
    cuenca = limpiar(cuenca)
    for row in c.fetchall():
        if cuenca == row[1]:
            existe =True
            idC = row[0]
            break
        idC = row[0]+1
    if existe == False:
        c.execute("INSERT INTO Cuenca(id, nombre) VALUES (?, ?)", (idC, cuenca))
        conn.commit()
    return idC

def insertarContrato(contrato):
    c.execute("SELECT * FROM Contrato")
    existe = False
    idCo = 0
    contrato = quitarAcento(contrato)
    contrato = limpiar(contrato)
    for row in c.fetchall():
        if contrato == row[1]:
            existe =True
            idCo = row[0]
            break
        idCo = row[0]+1
    if existe == False:
        c.execute("INSERT INTO Contrato(id, nombre) VALUES (?, ?)", (idCo, contrato))
        conn.commit()
    return idCo

def insertarData(arregloMes):
    c.execute("SELECT * FROM Data")
    rows = c.fetchall()
    idDa = len(rows)+1
    c.execute("INSERT INTO Data(id, enero, febrero, marzo, abril, mayo, junio, julio, agosto, septiembre, octubre, noviembre, diciembre) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", (idDa, arregloMes[0], arregloMes[1], arregloMes[2], arregloMes[3], arregloMes[4], arregloMes[5], arregloMes[6], arregloMes[7], arregloMes[8], arregloMes[9], arregloMes[10], arregloMes[11]))
    conn.commit()
    return idDa

def insertarCampoCrudo(idD, idM, idE, idC, idCo, idDa, campo, year):
    c.execute("SELECT * FROM Data")
    rows = c.fetchall()
    idCamp = len(rows)
    campo = quitarAcento(campo)
    campo = limpiar(campo)
    c.execute("INSERT INTO CampoCrudo(id, campo, year, departamentoID, municipioID, operadoraID, cuencaID, contradoID, datosID ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", (idCamp, campo, year, idD, idM, idE, idC, idCo, idDa))
    conn.commit()


In [23]:
def deleteTable():
    c.execute("DROP TABLE Data")
    c.execute("DROP TABLE CampoCrudo")
    c.execute("DROP TABLE Departamento")
    c.execute("DROP TABLE Municipio")
    c.execute("DROP TABLE Empresa")
    c.execute("DROP TABLE Cuenca")
    c.execute("DROP TABLE Contrato")
    conn.commit


In [24]:
def checkifwork():
    listnamefiles = [f for f in glob.glob("*.xlsx")]
    listnamefiles2 = [f for f in glob.glob("*.xls")]
    number = len(listnamefiles) + len(listnamefiles)
    print("please enter the number of Excelfiles there are in github, in order to check and save in the database")
    x = input()
    result = False
    if number == int(x):
        print("the download was a success")
        result = True
    else:
        print("error downloading the files from github, please download the files manually and put them in the same folder of this file ")
        for i in range(0,len(listnamefiles)):
            os.remove(listnamefiles[i])
        for i in range(0,len(listnamefiles2)):
            os.remove(listnamefiles2[i])
    return result

In [25]:
def deleteFiles(listname):
    for i in range(0, len(listname)):
        os.remove(listname[i])


In [26]:
def getFilesNamesGitHub ():
    url = 'https://github.com/dgguarin20/archivos'
    
    html = requests.get(url)
    with open('file.txt', 'wb') as wr: 
        wr.write(html.content)
    wr.close()

    read = open("file.txt","r+",encoding="utf-8")

    listhtmlExcel = []
    htmlExcel = ""
    for line in read:
        xls = ".xls"
        if xls in line:
            htmlExcel = line
            listhtmlExcel.append(htmlExcel)
    print(len(listhtmlExcel))
    listaExcel = []
    for j in range(0 , len(listhtmlExcel)):
        htmlExcel = listhtmlExcel[j]
        htmlExcel.replace(">", " ")
        htmlExcel.replace("<", " ")
        

        separacionExcel= np.array(htmlExcel.split(" "))
        for i in range(0,len(separacionExcel)):
            if "href" in separacionExcel[i]:
                l = separacionExcel[i].split("\"")
                urlgit = "https://github.com"+l[1]
                urlgit2 = urlgit.split("blob")
                urlgit = urlgit2[0]+"raw"+urlgit2[1]
                listaExcel.append(urlgit)
                
    read.close()
    listaExcel.append(" ")
    return listaExcel



In [30]:
def RunFile():
    print("WELCOME!")
    print("BEFORE STARTING CHECK YOU DONT HAVE ANY EXCEL FILES IN THIS FOLDER, ONCE YOU ARE READY CLICK ENTER")
    x = input()
    print("FIRST OF ALL LETS START WITH THE ANH FILES FROM THE WEBPAGE")
    listaExcel = getFilesNames()
    downloadFiles (listaExcel)
    listnameGas, listnameOil, listnameAll = getGasOilFileNames(True)
    print(".....................")
    print("CREATING THE DATABASE")
    print(".....................")
    create_tables()
    guardarDatosOil(listnameOil)
    print("FINISH DOWNLOADING ALL THE ANH FILES IN THE DATABASE")
    print("------------------------------------------------")
    print("DELETING THE EXCEL FILES")
    deleteFiles(listnameAll)
    print("------------------------------------------------")
    print("DOWNLOADING BLIND FILES FROM GITHUB:")
    namefilesGithub = getFilesNamesGitHub()
    downloadFiles(namefilesGithub)
    result = checkifwork()
    listnameGas, listnameOil, listnameAll = getGasOilFileNames(result)
     
    guardarDatosOil(listnameOil)
    print("THANK YOU FOR WAITING!")
    print("WE ARE DONE PLEASE RUN THE OTHER PYTHON FILES TO SEE THE RESULTS THANK YOU")
RunFile()

WELCOME!
BEFORE STARTING CHECK YOU DONT HAVE ANY EXCEL FILES IN THIS FOLDER, ONCE YOU ARE READY CLICK ENTER

FIRST OF ALL LETS START WITH THE ANH FILES FROM THE WEBPAGE
..................................................
Getting the name of the ANH files from the webpage
..................................................
downloading files... this might take some minutes
Download complete!
.....................
CREATING THE DATABASE
.....................
starting to save the data in the database this might take some minutes
saving:  Produccion-20Fiscalizada-20Crudo-2015.xlsx
finish with:  Produccion-20Fiscalizada-20Crudo-2015.xlsx
saving:  Produccion-20Fiscalizada-20Crudo-2018.xlsx
finish with:  Produccion-20Fiscalizada-20Crudo-2018.xlsx
saving:  Produccion-20Fiscalizada-20Crudo-2019-DIC.xlsx
finish with:  Produccion-20Fiscalizada-20Crudo-2019-DIC.xlsx
saving:  Produccion-20Fiscalizada-20Crudo-2020-20Agosto.xlsx
finish with:  Produccion-20Fiscalizada-20Crudo-2020-20Agosto.xlsx
saving:  P

please enter the number of Excelfiles there are in github, in order to check and save in the database
3
error downloading the files from github, please download the files manually and put them in the same folder of this file 
False
Please put the files in the same folder as this file in order to try it locally. once you do please write 1 and enter:
1
Files\
Files\*.xlsx
['Files\\Producción fiscalizada crudo 2016_18042018.xls']
starting to save the data in the database this might take some minutes
saving:  Files\Producción fiscalizada crudo 2016_18042018.xls


KeyboardInterrupt: 